# ЛР2 — трекинг объекта по ключевым точкам (новый ноутбук)

Свежая версия лабораторной работы без опоры на предыдущие блокноты. Используется классический подход: ключевые точки ORB, сопоставление `BFMatcher`, оценка гомографии через RANSAC и прорисовка рамки объекта на каждом кадре.


## Алгоритм
- Первый кадр видео берётся как шаблон объекта.
- Шаблон переводится в градации серого, по желанию применяется CLAHE.
- Строится пирамида шаблонов по масштабу, для каждой копии считаются ключевые точки и дескрипторы ORB.
- Для каждого кадра: детектируются признаки, сопоставляются с каждым масштабом шаблона (ratio test), по лучшей группе ищется гомография (RANSAC).
- Если число инлаеров достаточно — рисуется проецированный контур объекта и подпись; иначе выводится `Object not found`.


In [ ]:
import cv2
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple


In [ ]:
def make_orb(nfeatures: int = 2000, scale_factor: float = 1.2, nlevels: int = 8) -> cv2.ORB:
    """Конфигурация ORB с запасом по ключевым точкам."""
    return cv2.ORB_create(
        nfeatures=nfeatures,
        scaleFactor=scale_factor,
        nlevels=nlevels,
        edgeThreshold=15,
        patchSize=31,
    )


def preprocess_gray(frame: np.ndarray, use_clahe: bool = False) -> np.ndarray:
    """BGR -> серый; опционально CLAHE для подъёма контраста."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if use_clahe:
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
    return gray


def detect_features(detector: cv2.Feature2D, gray: np.ndarray) -> Tuple[List[cv2.KeyPoint], np.ndarray]:
    keypoints, descriptors = detector.detectAndCompute(gray, None)
    if descriptors is None:
        descriptors = np.zeros((0, detector.descriptorSize()), dtype=np.uint8)
    return keypoints, descriptors


def build_template_pyramid(template_gray: np.ndarray, detector: cv2.Feature2D, scales: Tuple[float, ...]) -> List[Dict]:
    """Создаёт набор шаблонов разных масштабов со своими признаками."""
    templates = []
    for scale in scales:
        resized = cv2.resize(template_gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        kp, desc = detect_features(detector, resized)
        if len(kp) == 0 or len(desc) == 0:
            continue
        h, w = resized.shape[:2]
        corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
        templates.append({"scale": scale, "kp": kp, "desc": desc, "corners": corners})
    return templates


def match_with_ratio(matcher: cv2.BFMatcher, desc1: np.ndarray, desc2: np.ndarray, ratio: float) -> List[cv2.DMatch]:
    if len(desc1) == 0 or len(desc2) == 0:
        return []
    knn = matcher.knnMatch(desc1, desc2, k=2)
    good = [m for m, n in knn if m.distance < ratio * n.distance]
    return good


In [ ]:
def track_object(
    video_path: str,
    output_path: str,
    *,
    ratio: float = 0.78,
    min_matches: int = 12,
    min_inliers: int = 10,
    scales: Tuple[float, ...] = (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4),
    use_clahe: bool = True,
    verbose: bool = False,
) -> str:
    """Трекинг объекта по первому кадру, результат сохраняется в mp4."""
    video_path = Path(video_path)
    output_path = Path(output_path)

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f"Не удалось открыть видео: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ok, first_frame = cap.read()
    if not ok:
        cap.release()
        raise RuntimeError("Видео пустое — нет первого кадра")

    detector = make_orb()
    template_gray = preprocess_gray(first_frame, use_clahe)
    templates = build_template_pyramid(template_gray, detector, scales)
    if not templates:
        raise RuntimeError("Не нашли ключевые точки в шаблоне")

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    writer = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    frame_id = 0
    while True:
        if frame_id == 0:
            frame = first_frame
        else:
            ok, frame = cap.read()
            if not ok:
                break
        frame_gray = preprocess_gray(frame, use_clahe)
        kp_frame, desc_frame = detect_features(detector, frame_gray)
        if len(desc_frame) == 0:
            writer.write(frame)
            frame_id += 1
            continue

        best = {"inliers": 0, "corners": None}
        for tpl in templates:
            good = match_with_ratio(matcher, tpl["desc"], desc_frame, ratio)
            if len(good) < min_matches:
                continue
            src_pts = np.float32([tpl["kp"][m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 4.0)
            if H is None or mask is None:
                continue
            inliers = int(mask.sum())
            if inliers > best["inliers"]:
                corners = cv2.perspectiveTransform(tpl["corners"], H)
                best = {"inliers": inliers, "corners": corners}

        if best["corners"] is not None and best["inliers"] >= min_inliers:
            pts = np.int32(best["corners"])
            cv2.polylines(frame, [pts], True, (0, 180, 0), 3)
            x, y = pts[0, 0]
            label = f"Object ({best['inliers']} inliers)"
            cv2.putText(frame, label, (int(x), int(max(y - 10, 20))), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 220, 220), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Object not found", (24, 36), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        if verbose:
            cv2.putText(
                frame,
                f"kp frame: {len(kp_frame)}",
                (24, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.55,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )
        writer.write(frame)
        frame_id += 1

    cap.release()
    writer.release()
    return str(output_path)


## Запуск на тестовом видео `mona-lisa.avi`
Код ниже прогоняет трекер и сохраняет результат в `results/lr2_mona_lisa.mp4`.


In [ ]:
output_video = track_object(
    "mona-lisa.avi",
    "results/lr2_mona_lisa.mp4",
    ratio=0.78,
    min_matches=12,
    min_inliers=10,
    scales=(1.0, 0.85, 0.7, 0.55, 0.4),
    use_clahe=True,
)
print(f"Результат сохранён в: {output_video}")
